In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CauseML.parameters import build_parameters_from_specification, build_parameters_from_metric_levels
from CauseML.constants import Constants
from CauseML.data_generation import DataGeneratingProcessWrapper
import CauseML.data_sources as data_sources
from CauseML.utilities import extract_treat_and_control_data
from CauseML.data_metrics import calculate_data_metrics

In [3]:
from collections import defaultdict
from IPython.display import clear_output
import matplotlib.pyplot as plt

## 1. Demonstration of Basic Dataset Generation Workflow

The code below demonstrates how the various pieces of CauseML benchmarking work together to generate a synthetic dataset.

1. Covariate data is fetched from the random covariate data source (imported above).


2. Parameters are generated by setting some metrics to given levels (with the rest left at default).


3. A data generating process is sampled according to the parameters and this DGP is used to generate data.


4. The observed and oracle data is extracted from the DGP wrapper. The observed data corresponds to the data a model would run on: the observed covariates, treatment assignment and outcome. The oracle data contains the potential outcomes and the transformed covariates which make up the treatment and outcome functions. Throughout this notebook, these transformed covariates are referenced as the 'True Covariates'. This is because a simple linear model over these values would accurately model the treatment and response surfaces.

In [33]:
covar_data_source = data_sources.load_random_normal_covariates(n_covars = 5, n_observations=100)
covar_data_source = data_sources.load_cpp()

In [34]:
covar_data = covar_data_source.get_data()
print(covar_data.shape)
covar_data.head()

(4802, 82)


,x_1,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,...,x_21_L,x_21_M,x_21_N,x_21_O,x_21_P,x_24_A,x_24_B,x_24_C,x_24_D,x_24_E
0,-0.117647,-0.966667,-0.6,0.052632,-0.871560,-1.0,-1.0,-0.6,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.235294,-1.000000,-1.0,0.263158,-0.018349,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.235294,-1.000000,-1.0,0.052632,-0.715596,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.352941,-1.000000,-1.0,0.315789,-0.055046,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.411765,-0.333333,-0.2,0.210526,-0.467890,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
dgp_params = build_parameters_from_metric_levels({
    Constants.MetricNames.TREATMENT_NONLINEARITY: Constants.MetricLevels.HIGH,
    Constants.MetricNames.TE_HETEROGENEITY: Constants.MetricLevels.LOW
})

In [39]:
%%time

dgp_wrapper = DataGeneratingProcessWrapper(
    parameters=dgp_params, data_source=covar_data_source)

dgp_wrapper.sample_dgp()

CPU times: user 10.9 s, sys: 0 ns, total: 10.9 s
Wall time: 10.9 s


In [37]:
%%time

_ = dgp_wrapper.generate_data()

CPU times: user 1.52 s, sys: 0 ns, total: 1.52 s
Wall time: 1.52 s


In [9]:
obs = dgp_wrapper.get_observed_data()
oracle = dgp_wrapper.get_oracle_data()

In [10]:
obs.head()

,T,Y,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,NOISE(Y)
86,1,0.790360,0.615385,2.0,-1.0,-1.000000,0.263158,0.000000,-0.5,-1.000000,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,1.0,-0.818182,0.777778,-0.414
26,0,-1.519622,-0.230769,2.0,0.0,0.043478,0.052632,-0.446154,-1.0,-0.333333,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,0.0,0.090909,-0.111111,-0.045
167,0,1.328515,0.153846,2.0,-1.0,0.304348,0.157895,0.461538,-0.5,-1.000000,...,-1.0,-0.6,-1.000000,-1.0,-1.0,-1.0,0.0,-0.636364,0.333333,-0.267
85,1,-1.863384,-0.230769,2.0,-0.5,0.043478,0.368421,0.076923,-1.0,-1.000000,...,-1.0,-1.0,0.333333,-1.0,-1.0,-1.0,0.0,-0.272727,0.333333,-0.370
67,0,-1.120737,0.615385,2.0,-0.8,0.913043,0.473684,0.830769,-1.0,-1.000000,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,0.0,-0.090909,1.000000,0.147


In [11]:
oracle.head()

,logit(P(T|X)),P(T|X),Y0,Y1,TE,TRANSFORMED_X0,TRANSFORMED_X1,TRANSFORMED_X2,TRANSFORMED_X3,TRANSFORMED_X4,...,TRANSFORMED_X65,TRANSFORMED_X66,TRANSFORMED_X67,TRANSFORMED_X68,TRANSFORMED_X69,TRANSFORMED_X70,TRANSFORMED_X71,TRANSFORMED_X72,TRANSFORMED_X73,TRANSFORMED_X74
86,2.197225,0.900000,1.644360,0.790360,-0.854,-0.070636,0.220909,-0.000000,0.000000,-0.019109,...,0.146000,0.152,-0.000000,-0.404000,-0.003421,0.344455,0.092556,-0.037864,-0.000615,0.104727
26,-0.390147,0.403682,-1.519622,-2.373622,-0.854,-0.001121,0.000000,-0.001086,0.006444,0.001433,...,-0.006348,0.000,0.002254,0.017565,0.000000,0.001664,0.000000,-0.001202,0.000231,-0.011636
167,0.345907,0.585625,1.328515,0.474515,-0.854,-0.014127,0.171818,0.007866,0.020000,-0.002866,...,-0.044435,0.000,-0.006996,0.122957,-0.002053,-0.081538,0.039667,-0.012621,-0.000092,0.048873
85,-0.666461,0.339290,-1.009384,-1.863384,-0.854,-0.010091,0.036818,0.000187,0.003333,0.010032,...,-0.006348,0.000,-0.002721,0.017565,-0.004789,-0.004992,0.019833,-0.010818,0.000231,0.034909
67,-0.726263,0.326015,-1.120737,-1.974737,-0.854,-0.010091,0.009818,0.042478,0.054000,-0.017198,...,-0.133304,0.000,-0.037778,0.184435,-0.009853,-0.034945,0.095200,-0.010818,-0.000615,0.005818


## 2. Metrics and Measures

The code below displays the various metrics which specify the distributional challenges in the data and the measures used to quantify them.

Note that most of the measures below are what is referred to as Oracle measures. They are not values we could calculate in real observational data as they rely on access to data about the counterfactual outcome and assignment mechanism.

#### 1. Outcome Nonlinearity

This is the degree of non-linearity in the outcome mechanism. It is measured as the linear fit $R^2$ between the observed covariates and the observed outcome. The higher the $R^2$, the more linear the outcome surface must be. This is not an oracle measure.

#### 2. Treatment Nonlinearity

This is the degree of non-linearity in the treatment mechanism. It is measured as the linear fit $R^2$ between the observed covariates and the logit of the propensity score. The higher the $R^2$, the more linear the treatment surface must be. This is an oracle measure as the propensity score and its logit is unknown in normal circumstances.

#### 3. Percent Treated

This is the percent of the dataset in the treated condition. It is a simple non-oracle measure.

#### 4. Balance

Three measures are used to quantify balance.

The simplest is the distance between the means of the true covariates in the two groups (see above for a definition of the true covariates). If the distributions are identical, the means will be in the same location. The problem with this measure is that it is sensitive to outliers or dimensions with higher variance (which can pull the mean far apart even if most dimensions have very similar values). This is a non-oracle metric.

The second simplest measure is the naive error in the average treatment effect estimate. In balance data, the ATE should be accurate. In imbalanced data it will not be. This is an oracle metric as it relies on access to the true ATE.

Finally, the Wassertein distance is used. This will be discussed more in future work but it is an integral probability measure of distance (the sum of the distance between the distribution CDFs across the domain) which means it is a pretty good measure of distance. 

#### 5. Alignment

The alignment measures the correlation ($R^2$) between the outcome and the the propensity logit. The higher this value, the greater the degree of confounding as the covariates which affect the outcome affect the assignment "more". This is an oracle metric as it relies on the propensity logit.

#### 6. Treatment Effect Heterogeneity

This measures the degree to which the treatment effect interacts with covariates (changes per individual). The standard deviation in the treatment effect is normalized by the standard deviation in the outcome so that the only variation measured is in the treatment effect itself (rather than changes coming from the base response function and not the interacion terms).


In [50]:
metrics_and_measures = {
#     Constants.MetricNames.OUTCOME_NONLINEARITY: [
#         "Lin r2(X_obs, Y)"
#     ],
    Constants.MetricNames.TREATMENT_NONLINEARITY: [
        "Lin r2(X_obs, Treat Logit)"
    ],
#     Constants.MetricNames.PERCENT_TREATED: [
#         "Percent(T==1)"
#     ],
#     Constants.MetricNames.BALANCE: [
# #         "Mean dist X_true: T=1<->T=0",
#         "Wass dist X_true: T=1<->T=0",
# #         "Naive TE"
#     ],
#     Constants.MetricNames.ALIGNMENT: [
#         "Lin r2(Y, Treat Logit)",
#         "Lin r2(Y0, Treat Logit)"
#     ],
#     Constants.MetricNames.TE_HETEROGENEITY: [
#         "std(TE)/std(Y)"
#     ]
}

levels = [
    Constants.MetricLevels.LOW,
    Constants.MetricLevels.MEDIUM,
    Constants.MetricLevels.HIGH,
]

## 3. Parameter Validation

The code below generates 30 datasets from 10 covariates for each possible setting (high, medium, low) of each metric and plots the distribution of the measures of each metric at each level. Each metric setting corresponds to different underlying parameters, so this serves as a validation that the parameter settings do indeed produce the desired effect on the relevant metric.

Each of the plots show a clear correspondence betweent the metric/parameter setting and the metric measures which track the realized level of the metric in the simulated data. The exact values of the metrics also agrees with the medians and quartiles stipulated in Dorie et al (2019)

There are two metrics worth exploring in a little more detail. Firstly, not that only the middle metric measure in Balance appears to accurately capture the increasing imbalance. Measuring balance is - in general - a hard problem. The Wassertein distance between the true covariates in the treat and control group is a promising measure based on the summed absolute error between the distribution CDFs. I plan to spend some time discussing this distance measure in the final draft.

Also note that the measure of treatment effect heterogeneity has a very wide distribution. This is because the distribution used to generate the treatment effect itself has thick tails. This means that there is large variance in the treatment effect size and hence large variance in the impact of the interactions with covariates. I plan to refine this soon.

In [ ]:
%%time

results = analyze_metric_measures_across_levels(
    metrics_and_measures,
#     lambda: data_sources.load_random_normal_covariates(n_covars = 10, n_observations=100),
#     data_sources.load_cpp,
    data_sources.load_lalonde,
    n_trials=50)


Running for TREATMENT_NONLINEARITY. Level:  LOW MEDIUM 

In [ ]:

plot_metric_measure_analysis(results, max_measure_count=3, mean=True, min_max=False)

In [43]:
def analyze_metric_measures_across_levels(
    metrics_and_measures, covar_generator,
    n_trials=20):
    '''
    Collect values for the given metrics and measures
    across all possible settings for each metric. Uses
    the gather_metric_measures_for_given_params to generate
    the metric measure values.
    '''

    results = defaultdict(lambda: defaultdict(dict))
    
    # Run for each given metric and set of measures
    for metric, measures in metrics_and_measures.items():
        print(f"\nRunning for {metric}. Level: ", end=" ")

        # Construct observation list of measures.
        observation_list = { metric: measures }

        # Run trials at all levels of metric.
        for level in levels:
            print(level, end=" ")
            dgp_params = build_parameters_from_metric_levels({
                metric: level
            })

            res = gather_metric_measures_for_given_params(
                    dgp_params, observation_list, covar_generator,
                    n_trials=n_trials)
            
            for measure, values in res[metric].items():
                results[metric][measure][level] = values
    
    return results

In [42]:
def gather_metric_measures_for_given_params(
        dgp_params, observation_spec,
        covar_generator,
        n_trials=10, verbose=False):
    
    '''
    Create n_trials datasets by sampling data generating processes according
    to the given dgp_params and data source parameters (n_covars, n_obs).
    
    Collect the metrics given in observation spec.
    '''
    results = defaultdict(lambda: defaultdict(list))
    for i in range(n_trials):
        if verbose:
            clear_output()
            print("Trials run:", i+1)
        results
        
        # Generate data
        data_source = covar_generator()
        
        # Sample DGP
        dgp_wrapper = DataGeneratingProcessWrapper(
            parameters=dgp_params, data_source=data_source)
        dgp_wrapper.sample_dgp()
        observed_covariate_data, observed_outcome_data, oracle_covariate_data, oracle_outcome_data = \
            dgp_wrapper.generate_data()
        
        # Calculate metrics
        metrics = calculate_data_metrics(
            observed_covariate_data, observed_outcome_data,
            oracle_covariate_data, oracle_outcome_data,
            observation_spec=observation_spec)
        
        # Build results
        for metric, measures in observation_spec.items():
            for measure in measures:
                res = metrics[metric][measure]
                if res is not None:
                    results[metric][measure].append(res)
    
    if verbose:
        for metric, measures in results.items():
            for measure, result_data in measures.items():
                print(f"{metric} {measure}:")
                print("min", round(np.min(result_data), 3), end=" ")
                print("mean:", round(np.mean(result_data), 3), end=" ")
                print("max", round(np.max(result_data), 3))
                print("-------------\n\n")
        
    return results

In [15]:
def plot_metric_measure_analysis(results, max_measure_count=1, mean=False, min_max=False):
    '''
    Plot the results of the analyze_metric_measures_across_levels
    function. Show the median, mean and 1st and 3rd quartiles for each
    metric measure at each level.
    '''
    
    level_colors = ["g", "b", "y"] # low medium high
    mean_color = "r" # mean will be red
    max_min_color = "k" # mean will be red
    for metric, measures in results.items():
        plt.figure(figsize=(10, 4))
        plt.suptitle(f"{metric}")
        plt.tight_layout()

        for measure_num, (measure_name, measure_values) in enumerate(measures.items()):
            plt.subplot(1, max_measure_count, 1 + measure_num)
            plt.title(f"{measure_name}")
            for level_num, level in enumerate(levels):
                level_values = measure_values[level]

                # Find quartile values in data
                quartiles = np.percentile(
                    level_values,
                    [25, 50, 75],
                    interpolation = 'midpoint')

                # Prepare plotting data
                x = level_num+1
                y_median = quartiles[1]
                
                
                err = np.array([
                    [y_median - quartiles[0]],
                    [quartiles[2] - y_median]
                ])

                # Plot
                color = level_colors[level_num]
                plt.xlim((0, max_measure_count+1))
                plt.scatter(x, y_median, label=level, color=color)
                plt.errorbar(x, y_median, err, color=color)
                
                if mean:
                    y_mean = np.mean(level_values)
                    plt.scatter(x, y_mean, color=mean_color)
                
                if min_max:
                    y_max = np.max(level_values)
                    y_min = np.min(level_values)
                    plt.scatter(x, y_max, color=max_min_color)
                    plt.scatter(x, y_min, color=max_min_color)
                
            plt.legend()

        plt.show()